# 1. 自动识别文件

In [1]:
import os
import datetime

In [2]:
list_file = list(os.walk('./'))[0][2]
for file in list_file:
    if 'Lennon_整理好的替代料总表' in file:
        alternate = file
    elif '改价表' in file:
        stock = file
    elif 'US MRP' in file:
        mrp = file

print(f'alternate: \t{alternate}')
print(f'stock: \t\t{stock}')
print(f'mrp: \t\t{mrp}')

alternate: 	Lennon_整理好的替代料总表_20230815_140740.xlsx
stock: 		改价表 8-18.xlsx
mrp: 		US MRP20230814-运营.xlsx


# 2. 计算库存
## 2.1 替代料库存

In [3]:
import pandas as pd

In [4]:
df_alternate = pd.read_excel(alternate,
                             header=0).fillna('')
df_alternate

,主料SKU,替换料1,替换料2,替换料3,替换料4,替换料5,替换料6
0,USCA171876-3,USCA171876D-3,USCA171876,USCA171876D,,,
1,USWM42150,USWM42150B,,,,,
2,USGS37022J,USGS37022T,USGS37022,USGS37022G,,,
3,USGS36761T,USGS36761,USGS36761TB,,,,
4,USGS41570B,USGS41570,,,,,
...,...,...,...,...,...,...,...
34440,USHHS41307-C,USHHS41307,,,,,
34441,USHHS41308-C,USHHS41308,,,,,
34442,USHHS94130,USHHS94130-C,,,,,
34443,USHHS94131,USHHS94131-C,,,,,


In [5]:
df_alternate['row'] = [str(i) for i in range(len(df_alternate))]

df_alternate

,主料SKU,替换料1,替换料2,替换料3,替换料4,替换料5,替换料6,row
0,USCA171876-3,USCA171876D-3,USCA171876,USCA171876D,,,,0
1,USWM42150,USWM42150B,,,,,,1
2,USGS37022J,USGS37022T,USGS37022,USGS37022G,,,,2
3,USGS36761T,USGS36761,USGS36761TB,,,,,3
4,USGS41570B,USGS41570,,,,,,4
...,...,...,...,...,...,...,...,...
34440,USHHS41307-C,USHHS41307,,,,,,34440
34441,USHHS41308-C,USHHS41308,,,,,,34441
34442,USHHS94130,USHHS94130-C,,,,,,34442
34443,USHHS94131,USHHS94131-C,,,,,,34443


In [6]:
df_stock = pd.read_excel(stock,
                         header=0,
                         names=['SKU', '当日库存'],
                         usecols='A, F')

df_stock

,SKU,当日库存
0,USBDA04239,3
1,USBDA04868,0
2,USDLA3ZP50,0
3,USEVC271FG,0
4,USBDA04007,1
...,...,...
240005,USGS49820,1
240006,USWR52564C,1
240007,USCVA63523-C,2
240008,USRAD13576-C,4


In [7]:
from tqdm import tqdm

In [8]:
for i in tqdm(range(len(df_stock)), desc='运行中', ncols=77):
    if df_stock.loc[i, 'SKU'].endswith('-NC'):
        df_stock = df_stock.drop(i)

运行中: 100%|█████████████████████| 240010/240010 [00:10<00:00, 22594.17it/s]


In [9]:
df_stock = df_stock.reset_index(drop=True)
df_stock['SKU'] = [df_stock.loc[i, 'SKU'][:-2] if df_stock.loc[i, 'SKU'].endswith('-N') else df_stock.loc[i, 'SKU'] for i in range(len(df_stock))]
df_stock = df_stock.groupby([df_stock['SKU']], as_index=False).sum()

df_stock

,SKU,当日库存
0,CAWR41163,1
1,CS20230609,0
2,CS230530,0
3,CSPJ20230615,1
4,CSPJ230609,0
...,...,...
72448,YHTWS58202-Y-6,0
72449,YHTWS5W1ZBA-Y-10,0
72450,YHTWS5W1ZBA-Y-8,0
72451,YHTWS802AA-Y-6,0


In [10]:
df_alternate['当日库存'] = 0
df_alternate = pd.merge(left=df_alternate, right=df_stock, how='left', left_on='主料SKU', right_on='SKU', suffixes=('', '_0')).fillna(0)
print('stock - 主料SKU')
df_alternate = pd.merge(left=df_alternate, right=df_stock, how='left', left_on='替换料1', right_on='SKU', suffixes=('', '_1')).fillna(0)
print('stock - 替换料1')
df_alternate = pd.merge(left=df_alternate, right=df_stock, how='left', left_on='替换料2', right_on='SKU', suffixes=('', '_2')).fillna(0)
print('stock - 替换料2')
df_alternate = pd.merge(left=df_alternate, right=df_stock, how='left', left_on='替换料3', right_on='SKU', suffixes=('', '_3')).fillna(0)
print('stock - 替换料3')
df_alternate = pd.merge(left=df_alternate, right=df_stock, how='left', left_on='替换料4', right_on='SKU', suffixes=('', '_4')).fillna(0)
print('stock - 替换料4')
df_alternate = pd.merge(left=df_alternate, right=df_stock, how='left', left_on='替换料5', right_on='SKU', suffixes=('', '_5')).fillna(0)
print('stock - 替换料5')
df_alternate = pd.merge(left=df_alternate, right=df_stock, how='left', left_on='替换料6', right_on='SKU', suffixes=('', '_6')).fillna(0)
print('stock - 替换料6')

df_alternate['当日库存'] = df_alternate['当日库存_0'] + df_alternate['当日库存_1'] + df_alternate['当日库存_2'] + df_alternate['当日库存_3'] + df_alternate['当日库存_4'] + df_alternate['当日库存_5'] + df_alternate['当日库存_6']
df_alternate = df_alternate[['主料SKU',
                             '替换料1',
                             '替换料2',
                             '替换料3',
                             '替换料4',
                             '替换料5',
                             '替换料6',
                             '当日库存']]

df_alternate

stock - 主料SKU
stock - 替换料1
stock - 替换料2
stock - 替换料3
stock - 替换料4
stock - 替换料5
stock - 替换料6


,主料SKU,替换料1,替换料2,替换料3,替换料4,替换料5,替换料6,当日库存
0,USCA171876-3,USCA171876D-3,USCA171876,USCA171876D,,,,12.0
1,USWM42150,USWM42150B,,,,,,0.0
2,USGS37022J,USGS37022T,USGS37022,USGS37022G,,,,18.0
3,USGS36761T,USGS36761,USGS36761TB,,,,,5.0
4,USGS41570B,USGS41570,,,,,,102.0
...,...,...,...,...,...,...,...,...
34440,USHHS41307-C,USHHS41307,,,,,,0.0
34441,USHHS41308-C,USHHS41308,,,,,,0.0
34442,USHHS94130,USHHS94130-C,,,,,,0.0
34443,USHHS94131,USHHS94131-C,,,,,,0.0


In [11]:
list_df = []
for i in ['主料SKU', '替换料1', '替换料2', '替换料3', '替换料4', '替换料5', '替换料6']:
    df_temp =  df_alternate[[i,
                             '当日库存']]
    df_temp.columns = ['SKU',
                       '当日库存']
    list_df.append(df_temp)

df_position = pd.concat(list_df)
df_position = df_position[df_position['SKU'] != ''].reset_index(drop=True)

df_position

,SKU,当日库存
0,USCA171876-3,12.0
1,USWM42150,0.0
2,USGS37022J,18.0
3,USGS36761T,5.0
4,USGS41570B,102.0
...,...,...
73404,USGS27875GB,3.0
73405,USGS16178NZ,0.0
73406,USGS6310NZ,27.0
73407,USIC2A367A,5.0


## 2.2 MRP表库存

In [12]:
df_mrp = pd.read_excel(mrp,
                       header=1,
                       usecols='B').fillna(0)

df_mrp

,MRP SKU
0,USTBC40007-C
1,USHB13158-2-C
2,USBM00237D-C
3,USBM00191D-C
4,USSGTO012T-C
...,...
55881,USWRM48301
55882,USWRM51020
55883,USWRM51021
55884,USWRM51022


In [13]:
df_mrp = pd.merge(left=df_mrp, right=df_stock, how='left', left_on='MRP SKU', right_on='SKU').fillna(0)
df_mrp = pd.merge(left=df_mrp, right=df_position, how='left', left_on='MRP SKU', right_on='SKU', suffixes=('_stock', '_position')).fillna('')

df_mrp['库存'] = [df_mrp.loc[i, '当日库存_position'] if df_mrp.loc[i, '当日库存_position'] != '' else df_mrp.loc[i, '当日库存_stock'] for i in range(len(df_mrp))]
df_mrp = df_mrp[['MRP SKU',
                 '库存']]

df_mrp

,MRP SKU,库存
0,USTBC40007-C,264.0
1,USHB13158-2-C,0.0
2,USBM00237D-C,57.0
3,USBM00191D-C,113.0
4,USSGTO012T-C,128.0
...,...,...
55881,USWRM48301,0.0
55882,USWRM51020,0.0
55883,USWRM51021,0.0
55884,USWRM51022,0.0


# 3. 保存

In [14]:
import datetime

In [15]:
writer = pd.ExcelWriter('Lennon_整理好的库存表_' + datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + '.xlsx')
df_mrp.to_excel(writer, index=False)
writer.close()